In [16]:
import sys
sys.path.append("../../models/Multi-Body")
from multidataset import MultiDataset
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from multimodel import Net as Model
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve
import matplotlib.pyplot as plt
import os
import json
from scipy import interpolate

In [17]:
all_models = [f for f in os.listdir('../../models/Multi-Body/trained_models') if '_best' in f]
print('\n'.join(all_models))

MultiBody8-Subjettiness_massTrue_ptTrue_best_ThreeLayer
MultiBody8-Subjettiness_massTrue_ptTrue_best_OneLayer400
MultiBody8-Subjettiness_massTrue_ptFalse_best_nojetpt
MultiBody8-Subjettiness_massTrue_ptTrue_best_OneLayer
MultiBody8-Subjettiness_massTrue_ptTrue_best_trial
MultiBody8-Subjettiness_massTrue_ptTrue_best_baseline
MultiBody8-Subjettiness_massTrue_ptTrue_best_TwoLayer
MultiBody8-Subjettiness_massTrue_ptTrue_best_tau_x_1


In [18]:
#Testing path
test_path = '../../datasets/n-subjettiness_data/test_all.npy'

In [19]:
def eval2(model, data):
    data = torch.tensor(data, dtype = torch.float32).cuda()
    return model.forward(data)

In [20]:
#Parameters to change, can change model_type to 'best' or 'last'
#all_models = ['MultiBody8-Subjettiness_massTrue_ptTrue_best_baseline', 'MultiBody8-Subjettiness_massTrue_ptTrue_best_tau_x_1']
for modelname in all_models:
    model_dict = json.load(open('../../models/Multi-Body/trained_model_dicts/' + modelname.replace("_best", "") + ".json"))
    use_jet_pt = model_dict['use_pt']
    use_jet_mass = model_dict['use_mass']
    tau_x_1 = model_dict['tau_x_1']
    N = model_dict['N']
    # model_type = 'best'
    extra_name = model_dict['label']
    if extra_name != '' and not extra_name.startswith('_'):
        extra_name = '_' + extra_name
    if tau_x_1 and 'tau_x_1' not in extra_name:
        extra_name += '_tau_x_1'
    hidden = list(map(int, model_dict['nodes'].split(',')))


    #model path
    model_path = '../../models/Multi-Body/trained_models/MultiBody' + str(N) + '-Subjettiness_mass' +\
                 str(use_jet_mass) + '_pt' + str(use_jet_pt) + '_best' + extra_name


    #loading model
    model = Model(N, use_jet_pt, use_jet_mass, tau_x_1, hidden).cuda().float()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    nparams = sum(p.numel() for p in model.parameters())
    #Loading testing dataset
    test_set = MultiDataset(test_path, N, use_jet_pt, use_jet_mass, tau_x_1)
    data,labels = test_set[:]
    with torch.no_grad():
        results = eval2(model,data).cpu().numpy()
        
    labels, results = labels[:,1], results[:,1]
    auc = roc_auc_score(labels, results)*100
    fpr, tpr, thresholds = roc_curve(labels, results)
    intrp = interpolate.interp1d(tpr, fpr)
    brr = 1./intrp(0.3)
    
    accuracy = accuracy_score(labels, results.round())*100
    #print("{}\t ROC-AUC: {:.4f}%, Accuracy: {:.4f}%".format(modelname, auc, accuracy))
    print("{} \t\t Params: {}\t ROC-AUC: {:.2f}%, Accuracy: {:.2f}%, BRR: {:.2f}".format(modelname,nparams,auc,accuracy,brr))

MultiBody8-Subjettiness_massTrue_ptTrue_best_ThreeLayer 		 Params: 54952	 ROC-AUC: 97.98%, Accuracy: 92.81%, BRR: 815.75
MultiBody8-Subjettiness_massTrue_ptTrue_best_OneLayer400 		 Params: 10002	 ROC-AUC: 97.87%, Accuracy: 92.68%, BRR: 636.49
MultiBody8-Subjettiness_massTrue_ptFalse_best_nojetpt 		 Params: 57302	 ROC-AUC: 97.97%, Accuracy: 92.80%, BRR: 774.64
MultiBody8-Subjettiness_massTrue_ptTrue_best_OneLayer 		 Params: 5002	 ROC-AUC: 97.84%, Accuracy: 92.63%, BRR: 577.62
MultiBody8-Subjettiness_massTrue_ptTrue_best_trial 		 Params: 57502	 ROC-AUC: 97.79%, Accuracy: 92.51%, BRR: 644.70
MultiBody8-Subjettiness_massTrue_ptTrue_best_baseline 		 Params: 57502	 ROC-AUC: 97.98%, Accuracy: 92.83%, BRR: 796.25
MultiBody8-Subjettiness_massTrue_ptTrue_best_TwoLayer 		 Params: 45202	 ROC-AUC: 97.97%, Accuracy: 92.79%, BRR: 774.64
MultiBody8-Subjettiness_massTrue_ptTrue_best_tau_x_1 		 Params: 54902	 ROC-AUC: 97.55%, Accuracy: 92.14%, BRR: 516.43
